In [38]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import egg.core as core
import torchvision

from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import random
import numpy as np
import string

from sklearn.model_selection import train_test_split
import itertools

from pylab import rcParams
rcParams['figure.figsize'] = 5, 10

# For convenince and reproducibility, we set some EGG-level command line arguments here
opts = core.init(params=['--random_seed=7', # will initialize numpy, torch, and python RNGs
                         '--lr=1e-3',   # sets the learning rate for the selected optimizer 
                         '--batch_size=32',
                         '--optimizer=adam'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEED = 7
random.seed(SEED)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# create inputs

# function to make the one hot vectors for each i_att
def generate_one_hot_vectors(x):
    one_hot_vectors = []
    for i in range(x):
        vec = np.zeros(x)  # Create a zero vector of length x
        vec[i] = 1         # Set the i-th position to 1
        one_hot_vectors.append(vec)
    return one_hot_vectors

# item is concat of its attributes, create all possible combos
def concat_one_hots(one_hot_vectors, num_attributes):
    combinations = []
    for combo in itertools.product(one_hot_vectors, one_hot_vectors):
        combined = np.concatenate(combo)  # Concatenate the two one-hot vectors
        combinations.append(combined)
    return combinations


def create_input(num_attributes, values):

    # Creating the encoder
    input = np.array([])

    if num_attributes == 'Random':
        num_attributes = np.random.randint(2, 5)
    elif isinstance(num_attributes, int) == "False":
        raise TypeError("Please enter an int for num_attributes.")
    elif num_attributes < 2 or num_attributes > 4:
        raise ValueError("Please choose value for n_attributes >= 2, <=4.")

    if values == 'Random':
        for attribute in range(num_attributes):
            values = np.random.randint(4, 101)
    elif isinstance(values, int) == "False":
        raise TypeError("Please enter an int between 4 and 100 for values.")
    elif values < 4 or values > 100:
        raise ValueError("Please choose an int  >=4, <=100.")

    one_hots = generate_one_hot_vectors(values)

    results = concat_one_hots(one_hots, num_attributes)
    size_i = values**(num_attributes)

    
    return results, size_i, values, num_attributes


In [40]:
dataset, size_data, n_values, n_attributes = (create_input("Random", "Random"))

dataset_size = len(dataset)
message_len = 10

# shuffle the input data
random.shuffle(dataset)

In [41]:
data_with_label = []
for i in range(len(dataset)):
    data_with_label.append([dataset[i], i])

In [42]:
print(data_with_label)

[[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]), 0], [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [43]:
# copied from code compo_vs_generalization

def split_train_test(dataset, p_hold_out=0.1, random_seed=7):

    assert p_hold_out > 0
    random_state = np.random.RandomState(seed=random_seed)

    n = len(dataset)
    permutation = random_state.permutation(n)

    n_test = int(p_hold_out * n)

    test = [dataset[i] for i in permutation[:n_test]]
    train = [dataset[i] for i in permutation[n_test:]]
    assert train and test

    assert len(train) + len(test) == len(dataset)

    return train, test

In [44]:
train, test = split_train_test(data_with_label)

In [45]:
#train_tensor = torch.tensor(np.array(train[1]).tolist())
#test_tensor = torch.tensor(np.array(test[1]).tolist())

In [46]:
""" # create alphabet 
alphabet_size = 100 

def create_alphabet(alphabet_size, seed):
    random.seed(seed)
    symbols = set()
    total_symbols = 63
    while len(symbols) != total_symbols:
        symbols.add(random.choice(string.ascii_letters + string.digits + "#"))


    if alphabet_size > total_symbols:
        alphabet = symbols.copy()
        while len(alphabet) != alphabet_size:
            alphabet.add(''.join(random.sample(sorted(symbols), 2)))

    return list(alphabet)


alphabet = create_alphabet(100, SEED) """

' # create alphabet \nalphabet_size = 100 \n\ndef create_alphabet(alphabet_size, seed):\n    random.seed(seed)\n    symbols = set()\n    total_symbols = 63\n    while len(symbols) != total_symbols:\n        symbols.add(random.choice(string.ascii_letters + string.digits + "#"))\n\n\n    if alphabet_size > total_symbols:\n        alphabet = symbols.copy()\n        while len(alphabet) != alphabet_size:\n            alphabet.add(\'\'.join(random.sample(sorted(symbols), 2)))\n\n    return list(alphabet)\n\n\nalphabet = create_alphabet(100, SEED) '

In [47]:
   
class Receiver(nn.Module):
    def __init__(self, hidden_size_receiver, n_inputs):
        super().__init__()
        self.fc = nn.Linear(hidden_size_receiver, n_inputs)
  
    def forward(self, rnn_output, _input = None):
        return self.fc(rnn_output)

In [48]:
# initialize parameters not sure for all of them what they do
n_output = 50
n_input = len(train)
max_len = 10
cell = 'lstm'
hidden_size = 142
embed_dim = 5
vocab_size = 10
temp = 1.0
hidden_size_receiver = 50

In [49]:
sender = nn.Linear(n_input, hidden_size)
sender = core.RnnSenderGS(sender, vocab_size=vocab_size, embed_dim=embed_dim, hidden_size=hidden_size, max_len=max_len, temperature=temp, cell=cell)

receiver = core.RnnReceiverGS(Receiver(hidden_size_receiver, n_input), vocab_size=vocab_size, embed_dim=embed_dim, hidden_size=hidden_size_receiver, cell='rnn')

In [50]:
def loss(sender_input, _message, _receiver_input, receiver_output, _labels):
    return (sender_input - receiver_output).pow(2.0).mean(dim=1), {'aux' : 0}

In [51]:
game = core.SenderReceiverRnnGS(sender, receiver, loss)

In [52]:
optimizer = torch.optim.Adam([
        {'params': game.sender.parameters(), 'lr': 1e-3},
        {'params': game.receiver.parameters(), 'lr': 1e-2}
    ])
callbacks = [core.ConsoleLogger(as_json=True)]

In [53]:
class TensorDataset(Dataset):
    def __init__(self, data):
        self.label = data[0][1]
        self.data = data[0][0]
        print(self.data)

    def __len__(self):
        return len(self.data)  # Return number of rows (lists)

    def __getitem__(self, idx):
        return self.data[idx], self.label  # Return the row (list) at the index

In [54]:
train_tensor_dataset = TensorDataset(train)
test_tensor_dataset = TensorDataset(test)

test_loader = DataLoader(test, batch_size=32, shuffle=False)
train_loader = DataLoader(train, batch_size=32,shuffle=False)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [56]:
trainer = core.Trainer(game=game, optimizer=optimizer, train_data=train_loader,
                           validation_data=test_loader)
trainer.train(15)

TypeError: Linear.forward() takes 2 positional arguments but 3 were given

In [ ]:
""" # parts adapted from compo_vs_generalization, parts adjusted for this case
class DiffLoss(torch.nn.Module):
    def __init__(self, n_attributes, n_values,
                 generalization=False):
        super().__init__()
        self.n_attributes = n_attributes
        self.n_values = n_values
        self.alphabet = alphabet
        self.message_len = message_len
        self.test_generalization = generalization

    def forward(self, sender_input, _message, _receiver_input, receiver_output, labels):
        batch_size = sender_input.size(0)
        sender_input = sender_input.view(
            batch_size, self.n_attributes, self.n_values)
        receiver_output = receiver_output.view(
            batch_size, self.n_attributes, self.n_values)
        
        acc = (torch.sum((receiver_output.argmax(dim=-1) == sender_input.argmax(dim=-1)
                              ).detach(), dim=1) == self.n_attributes).float().mean()
        acc_or = (receiver_output.argmax(dim=-1) ==
                      sender_input.argmax(dim=-1)).float().mean()
        
        flattened_tensor = torch.flatten(sender_input[0])
        probabilities = F.softmax(flattened_tensor / 1.0, dim=0)
        sampled_idx = torch.multinomial(probabilities, message_len)
        labels = ''
        for char in range(len(sampled_idx)):
            sampled_char = alphabet[sampled_idx[char]]
            labels += sampled_char
        loss = F.cross_entropy(receiver_output, labels, reduction="none").view(
                batch_size, self.n_attributes).mean(dim=-1)

        return loss, {'acc': acc, 'acc_or': acc_or}
        
 """

In [ ]:
""" #loss = DiffLoss(n_attributes, n_values)

def loss(sender_input, _message, _receiver_input, receiver_output, _labels):
    return F.mse_loss(sender_input, receiver_output, reduction='none').mean(dim=1), {'aux': 5.0} """

In [ ]:
""" train_loader = DataLoader(train, batch_size=alphabet_size)
test_loader = DataLoader(test, batch_size=alphabet_size) """

In [172]:
#trainer_labels = []
#for item in train_loader:
    #output_sender = sender(item[0][0])
    #flattened_tensor = torch.flatten(output_sender[0])
    #probabilities = F.softmax(flattened_tensor / 1.0, dim=0)
    #sampled_idx = torch.multinomial(probabilities, message_len, replacement=True)
    #label = ''
    #for char in range(len(sampled_idx)):
        #sampled_char = alphabet[sampled_idx[char]]
        #label += sampled_char
    #trainer_labels.append(label)


In [ ]:
""" game = core.SenderReceiverRnnReinforce(sender, receiver, loss, sender_entropy_coeff=0.0, receiver_entropy_coeff=0.0)
optimizer = torch.optim.Adam(game.parameters())

trainer = core.Trainer(
    game=game, optimizer=optimizer, train_data=train_loader,
    validation_data=test_loader
    ) """

In [ ]:
""" n_epochs = 15

input = torch.zeros((hidden_size, max_len)).normal_()
optimized_loss, aux_info = game(train, labels=None)

trainer.train(n_epochs) """